In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy import signal
from statistics import mode

In [ ]:
def get_mode_lastx(data, i, n=500):
    if i - n < 0:
        return mode(data[:i])
    else:
        return mode(data[i-n:i])



In [ ]:
def denoise_rssi(rssi, threshold=-15):
    new_rssi = np.copy(rssi)
    i = 1
    n_samples = len(new_rssi)
    
    while i < n_samples:
        delta = new_rssi[i] - new_rssi[i-1]
        
        if delta < threshold:
            j = i
            pre = new_rssi[i-1]
            #pre = get_mode_lastx(new_rssi, i-1)
            val = new_rssi[i]
            
            while new_rssi[j] == val or new_rssi[j] < pre:
                new_rssi[j] = pre
                j += 1
                
                if j==n_samples:
                    break
            
            i = j
            
        i += 1
            
    return new_rssi



In [ ]:
f = open('rssi_22', "r")
rssi = np.fromfile(f, dtype=np.int32)

denoised_rssi = denoise_rssi(rssi)

plt.figure(figsize=(16,6))
plt.plot(rssi, label='raw')
plt.plot(denoised_rssi, 'r', label='denoised')
plt.title("RSSI")
plt.xlabel("time [ms]")
plt.ylabel("rssi [dB]")
plt.legend()
plt.show()

In [ ]:


window_lenght = 799
polyorder = 1
filtered = signal.savgol_filter(denoised_rssi, window_lenght, polyorder)

s = 0
e = -1

plt.figure(figsize=(16,6))
plt.plot(rssi[s:e])
plt.plot(filtered[s:e], 'r')
plt.title("RSSI filtered using Savitzky-Golay filter (window={}, order={})".format(window_lenght, polyorder))
plt.xlabel("time [ms]")
plt.ylabel("rssi [dB]")
plt.show()



In [ ]:


factor = 13
downsampled = signal.decimate(filtered, factor)

plt.figure(figsize=(16,6))
plt.plot(downsampled, label='downsampled')
plt.title("Downsampling signal")
plt.xlabel("samples")
plt.ylabel("rssi [dB]")
plt.show()



In [ ]:
kernel_size = 149
filtered2 = signal.medfilt(downsampled, kernel_size)

plt.figure(figsize=(16,6))
plt.plot(downsampled[s:e], label='denoised')
plt.plot(filtered2[s:e], 'r', label='filtered')
plt.title("Re-filtered using Median filter (kernel={})".format(kernel_size))
plt.xlabel("time [ms]")
plt.ylabel("rssi [dB]")
plt.show()

In [ ]:
def process_rssi(rssi):
    denoised_rssi = denoise_rssi(rssi)
    filtered = signal.savgol_filter(denoised_rssi, 799, 1)
    downsampled = signal.decimate(filtered, 13)
    filtered2 = signal.medfilt(downsampled, 149)
    return filtered2

In [ ]:
for i in range(20):
    
    filename = "rssi_00 + str(i)
    f = open(filename, "r")
    rssi = np.fromfile(f, dtype=np.int32)
    processed = process_rssi(rssi)

    fig, axs = plt.subplots(2, figsize=(16,8))
    fig.suptitle('RSSI processing data{}'.format(i))
    axs[0].plot(rssi, label='raw')
    axs[1].plot(processed, 'r', label='processed')

